![Data_Layer](img\Data.png)

# 1. Data Layer – Preprocessing & Feature Engineering

This layer serves as the foundation of the project by preparing and transforming raw TCS stock data into a clean, enriched format suitable for exploratory data analysis and machine learning.

---

### Datasets Loaded
- `TCS_stock_history.csv` – Historical OHLCV data (Open, High, Low, Close, Volume)
- `TCS_stock_action.csv` – Corporate actions (e.g., splits, dividends)
- `TCS_stock_info.csv` – Stock-level metadata and reference information

---

### Data Cleaning & Date Parsing
- Converted `Date` columns to datetime objects.
- Removed rows with missing or null values across all datasets.
- Extracted date-related features: **Year, Month, Quarter, Day, Weekday**.

---

### Feature Engineering
- **Daily Return:** Day-to-day percentage change in Close price.
- **Cumulative Return:** Compounded product of daily returns to show investment growth.
- **MACD & Signal Line:** Technical indicators used to analyze momentum shifts.
- **RSI (Relative Strength Index):** Helps identify overbought/oversold market conditions.
- **52W High:** Highest stock price in the last 252 trading days (approx. 1 year).

---

### Lag Features & Rolling Statistics (for ML models)
- `Lag_1`, `Lag_2`: Previous Close prices (1-day and 2-day lag).
- `Rolling_Mean_7`: 7-day rolling average of Close price.
- `Rolling_Std_7`: 7-day rolling standard deviation of Close price.

These features are essential for time series regression and forecasting models like Random Forest, XGBoost, and LSTM.

---

### Outputs Saved
- `cleaned_TCS_stock_history.csv`
- `cleaned_TCS_stock_action.csv`
- `cleaned_TCS_stock_info.csv`

These files are now enriched, cleaned, and ready for downstream EDA and modeling.

---


In [ ]:
import pandas as pd
import numpy as np

# 📥 Load datasets
history_df = pd.read_csv("data/TCS_stock_history.csv")
action_df = pd.read_csv("data/TCS_stock_action.csv")
info_df = pd.read_csv("data/TCS_stock_info.csv")

# 📅 Convert Date columns
history_df['Date'] = pd.to_datetime(history_df['Date'], errors='coerce')
action_df['Date'] = pd.to_datetime(action_df['Date'], errors='coerce')

# 📆 DateTime features
history_df['Year'] = history_df['Date'].dt.year
history_df['Month'] = history_df['Date'].dt.month
history_df['Quarter'] = history_df['Date'].dt.quarter
history_df['Day'] = history_df['Date'].dt.day
history_df['Weekday'] = history_df['Date'].dt.day_name()

# ─────────────────────────────────────────────
# 📈 Feature Engineering

# Daily Return & Cumulative Return
history_df['Daily Return'] = history_df['Close'].pct_change()
history_df['Cumulative Return'] = (1 + history_df['Daily Return']).cumprod()

# MACD & Signal Line
ema_short = history_df['Close'].ewm(span=12, adjust=False).mean()
ema_long = history_df['Close'].ewm(span=26, adjust=False).mean()
history_df['MACD'] = ema_short - ema_long
history_df['Signal_Line'] = history_df['MACD'].ewm(span=9, adjust=False).mean()

# RSI
delta = history_df['Close'].diff()
gain = np.where(delta > 0, delta, 0)
loss = np.where(delta < 0, -delta, 0)
avg_gain = pd.Series(gain).rolling(window=14).mean()
avg_loss = pd.Series(loss).rolling(window=14).mean()
rs = avg_gain / avg_loss
history_df['RSI'] = 100 - (100 / (1 + rs))

# 52-Week High (approx. 252 trading days)
history_df['52W_High'] = history_df['High'].rolling(window=252).max()


# ─────────────────────────────────────────────
# 🔁 Lag Features & Rolling Statistics (for XGBoost)
history_df['Lag_1'] = history_df['Close'].shift(1)
history_df['Lag_2'] = history_df['Close'].shift(2)
history_df['Rolling_Mean_7'] = history_df['Close'].rolling(7).mean()
history_df['Rolling_Std_7'] = history_df['Close'].rolling(7).std()

# 🧹 Final Cleaning
history_df.dropna(inplace=True)
action_df.dropna(inplace=True)
info_df.dropna(how='all', inplace=True)

# 🧪 Prepare data for ML (optional here but useful in model layer)
features = [
    'Open', 'High', 'Low', 'Volume',
    'MACD', 'Signal_Line', 'RSI',
    'Lag_1', 'Lag_2', 'Rolling_Mean_7', 'Rolling_Std_7'
]
X = history_df[features]
y = history_df['Close']

# 💾 Save cleaned & engineered data
history_df.to_csv("data/cleaned_TCS_stock_history.csv", index=False)
action_df.to_csv("data/cleaned_TCS_stock_action.csv", index=False)
info_df.to_csv("data/cleaned_TCS_stock_info.csv", index=False)

# ✅ Preview
history_df.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Year,Month,...,Weekday,Daily Return,Cumulative Return,MACD,Signal_Line,RSI,Lag_1,Lag_2,Rolling_Mean_7,Rolling_Std_7
13,2002-08-29,30.402116,31.136377,30.299877,30.588005,1236544,0.0,0.0,2002,8,...,Thursday,0.000000,1.036209,0.330097,0.004831,55.849459,30.588005,30.560106,30.376876,1.037554
14,2002-08-30,30.680939,33.552919,30.671644,32.753597,3762384,0.0,0.0,2002,8,...,Friday,0.070799,1.109572,0.545634,0.112992,64.309181,30.588005,30.588005,30.992967,0.974951
15,2002-09-02,33.274077,34.184930,32.753588,33.088188,2389568,0.0,0.0,2002,9,...,Monday,0.010215,1.120906,0.734975,0.237388,67.659186,32.753597,30.588005,31.481587,1.054035
16,2002-09-03,33.274091,33.413508,31.229314,31.536030,986784,0.0,0.0,2002,9,...,Tuesday,-0.046910,1.068325,0.751125,0.340136,70.517293,33.088188,32.753597,31.493539,1.054123
17,2002-09-04,31.136362,32.530527,30.773879,32.223812,1449856,0.0,0.0,2002,9,...,Wednesday,0.021809,1.091624,0.810083,0.434125,72.285289,31.536030,33.088188,31.619678,1.085177


---